In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

#### Objective:
Predict **next 7 days Tesla (TSLA) daily closing prices** using historical daily candle close data.

In [10]:
stock_data=yf.download('TSLA',start='2010-06-29')


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['TSLA']: TypeError("'NoneType' object is not subscriptable")


In [11]:
print("Starting Date:",start)

NameError: name 'start' is not defined

In [12]:
stock_data.head(10)

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA,TSLA
Date,,,,,,


In [13]:
stock_data.tail(10)

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA,TSLA
Date,,,,,,


In [14]:
stock_data.describe()

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA,TSLA
count,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN


### Insights

- Price ranged from **~$1 to ~$490**, showing massive long-term growth.
- **75% of data lies below ~$210**, meaning recent highs dominate the averages.
- **Median close (~$20)** is far below the mean, indicating a strong **right-skew** due to recent price surge.
- **Trading volume is very high and volatile**, reflecting strong market interest.


In [6]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3922 entries, 2010-06-29 to 2026-01-30
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (Close, TSLA)   3922 non-null   float64
 1   (High, TSLA)    3922 non-null   float64
 2   (Low, TSLA)     3922 non-null   float64
 3   (Open, TSLA)    3922 non-null   float64
 4   (Volume, TSLA)  3922 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 183.8 KB


In [7]:
df = pd.DataFrame(stock_data)

In [8]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

KeyError: 'Date'

In [ ]:
df.rename(columns={
    "Price": "Open",  # if you want the first column as Open price
    "Close": "Close",
    "High": "High",
    "Low": "Low",
    "Open": "Volume"   # last column becomes Volume
}, inplace=True)